# Importar Librerias

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


# Carga de los dataset

In [3]:
df_21_23 = pd.read_csv('https://raw.githubusercontent.com/MaricelSantos/Mentoria-Diplodatos-2024/main/Conexiones_Transparentes_21-23.csv')
df_22_sys=  pd.read_csv('https://raw.githubusercontent.com/MaricelSantos/Mentoria-Diplodatos-2024/main/Conexiones_Transparentes.csv')

# Eliminacion de filas y columnas

De esto hablamos durante la última reunión. Definir criterios y aplicar el codigo necesario

# Separación del data set
Se los muestro de forma generica. Pero como hablamos nuestra variable target seria ICA el indice de calidad de agua.

In [ ]:
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Definición de Columnas y Transformaciones
Es necesario identificar las columnas numéricas y categóricas porque vamos a tener diferentes transformaciones según cada tipo de columna.


numeric_transformer contiene un pipeline con dos pasos, imputar que utiliza el metodo SimpleImputer con la mediana como argumento y escalar que utiliza un estandar escalar. categorial_transformer hace lo propio con las categoricas y luego preprocessor agrupa los transformadores para las columnas. Hasta aca tenemos de forma ordenada los procedimientos para cada columna.

In [ ]:
numeric_features = ['num_col1', 'num_col2']
categorical_features = ['cat_col1', 'cat_col2']

numeric_transformer = Pipeline(steps=[
    ('imputar', SimpleImputer(strategy='median')),
    ('scalar', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputar', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


# Definición de Modelos (TP3)
Esto es parte del TP3 pero se los muestro para que se entienda como se usa el pipeline. En este caso el ejemplo es con un Random Forest Regressor

In [ ]:
# Pipeline que incluye el preprocesamiento y el modelo RandomForestRegressor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))])


# Utilización del pipeline

In [ ]:

#Entrenamiento

pipeline.fit(X_train, y_train)

#Predicción

y_pred = pipeline.predict(X_test)

#Metricas

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Este ejemplo muestra cómo construir un pipeline para un problema de regresión utilizando RandomForestRegressor. Se pueden modificar las transformaciones, las estrategias de imputación y los hiperparámetros del modelo. En el TP3 va a ser necesario aplicar técnicas avanzadas como la búsqueda de hiperparámetros (GridSearchCV) o validación cruzada (cross-validation) en el modelo para optimizar aún más.

# Curación y Creación de Múltiples Versiones del Dataset

Vamos a ver el ejemplo de quitar outliers y comparar con un dataset donde no fueron removidos.

In [ ]:
def remove_outliers(df, columns):
    for col in columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        df = df[(df[col] >= (q1 - 1.5 * iqr)) & (df[col] <= (q3 + 1.5 * iqr))]
    return df

#Generamos el df sin outliers
df_no_outliers = remove_outliers(df, numeric_features)

#Separamos el df sin outliers
X_train_no_outliers, X_test_no_outliers, y_train_no_outliers, y_test_no_outliers = train_test_split(
    df_no_outliers.drop('target', axis=1), df_no_outliers['target'], test_size=0.2, random_state=42)

# Uso del pipeline comparativamente

In [ ]:
# Original Dataset
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
print(f'R^2 Score: {r2_score(y_test, y_pred)}')


# No Outliers Dataset
pipeline.fit(X_train_no_outliers, y_train_no_outliers)
y_pred_no_outliers = pipeline.predict(X_test_no_outliers)
print(f'No Outliers Dataset- Mean Squared Error: {mean_squared_error(y_test_no_outliers, y_pred_no_outliers)}')
print(f'No Outliers Dataset - R^2 Score: {r2_score(y_test_no_outliers, y_pred_no_outliers)}')


Lo que hace el pipeline es ahorrarnos bastante codigo y trabajar de una manera más limpia. Si no lo utilizamos un paso de normalización se vería así.

#SIN PIPELINE

In [4]:
#Defino el transformados que se va aplicar
scaler = StandardScaler()

#Aplico transformador a las numericas del train del dataset origical

X_train_normal = scaler.fit_transform(X_train[numeric_features])

# El paso anterior devuelve un array por lo tanto es necesario volver a obtener panda df
df_normalizado_train = pd.DataFrame(X_train_normal, columns=X_train[numeric_features].columns)

# En este paso se concatena lo obtenido con lo original o con lo que venga trabajando
# por ejemplo concatenar lo aplicado a las numericas con lo aplicado a las categoricas
X_train_normal_hotencoding = pd.concat([df_normalizado_train, df_hotencoding_train], axis=1)

#Repito los pasos con el test pero con el cuidado de que aquí no utilizo fit_transform
#solo transform porque se trata de test

X_test_normal = scaler.transform(X_test[numeric_features])
df_normalizado_test = pd.DataFrame(X_test_normal, columns=X_test[numeric_features].columns)
X_test_normal_hotencoding = pd.concat([df_normalizado_test, df_hotencoding_test], axis=1)


# Aplicación de un modelo
# Crear una instancia de RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
print(f'R^2 Score: {r2_score(y_test, y_pred)}')

NameError: name 'X_train' is not defined

El bloque anterior solo esta hecho para el dataset original. Si quiero comparar con el dataset sin outliers tengo que definir un bloque similar pero con las variables correspondientes a ese dataset. Luego puedo aplicar el modelo sobre los train y predecir sobre los test para comparar metricas.
Durante el cursado se ve el uso de transformadores de esta manera y no se ve pipelines por lo tanto es una decisión de ustedes que herramienta utilizar. Ambas son correctas